**Beginner's gradual improvement from 0.909 to 0.779 (English / 日本語)**

This is the first time for me to participate the Competition after joining Kaggle 2 months ago. I intend to accumulate my experiences by joinnig anything. Below is the record of gradual score improvement which a beginner like me made after trial and error.

Kaggleを始めて2か月、始めてCompetitionに参加してみる。Titanicも決して上位ではないが、とにかく色々参加して経験値を積んでいこうと思う。初心者が四苦八苦して、パブリックスコアを0.909→0.900→0.885→0.860→0.798→0.779へと、徐々に改善していった記録です。

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import nltk
from nltk.tokenize import word_tokenize

In [ ]:
train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
all_df = pd.concat([train, test], sort = False).reset_index(drop=True)

In [ ]:
train['excerpt_len'] = train['excerpt'].apply(lambda x : len(x))
train['excerpt_wordCnt'] = train['excerpt'].apply(lambda x : len(x.split(' ')))
train['excerpt_sentCnt'] = train['excerpt'].apply(lambda x : len(x.split('.')))
train["mean_WordPerSent"] = train["excerpt_wordCnt"]/train["excerpt_sentCnt"]
train["mean_LenPerWord"] = train["excerpt_len"]/train["excerpt_wordCnt"]

When think about "what is readability?", what I came up with at first was "less sentences, short sentences, less words". Based on this easy idea, tried to use number of words/sentences which were created by easy method, as well as average lengths...


「読みやすさとは何か？」を考えたとき、まず思いつくのは「文章が少ない、各文が短い、単語が少ない」ということかな？ということで、文字数・単語数・文章数」を特徴量として出してみる。単語数・文章数は簡易的にスペース・ピリオドで区切った数とする。それらを割り算して文章当たりの平均単語数・単語当たりの文字数も出してみる。

In [ ]:
X_train = train.drop(["url_legal","license","target","excerpt","standard_error"],axis=1)
Y_train = train["target"]

X_train["id"] = X_train["id"].astype("category")
X_train.head()

From a quick look, it seems 'not so bad', and try to submit using these features as first try. I'll try to use LightGBM which I'm familiar with, after cross-validation.


まずまずのようにも見えるので、とりあえずこれらの特徴量を使って最初のSubmitをしてみる。使い慣れたLightGBMで、クロスバリデーションをした後、予測してみる。

In [ ]:
kf = KFold(n_splits = 3)
models = []
rmses =[]
lgbm_params ={"objective":"regression", "random_seed":1234}

for train_index, val_index in kf.split(X_train):
    XX_train = X_train.iloc[train_index]
    XX_valid = X_train.iloc[val_index]
    YY_train = Y_train.iloc[train_index]
    YY_valid = Y_train.iloc[val_index]
    
    lgbm_train = lgbm.Dataset(XX_train, YY_train)
    lgbm_eval = lgbm.Dataset(XX_valid, YY_valid)
    
    model_lgbm = lgbm.train(lgbm_params,
                           lgbm_train,
                           valid_sets = lgbm_eval,
                           num_boost_round = 100,
                           early_stopping_rounds = 20,
                           verbose_eval = 10,
                           )
    y_pred = model_lgbm.predict(XX_valid, num_iteration = model_lgbm.best_iteration)
    tmp_rmse = np.sqrt(mean_squared_error(YY_valid, y_pred))
    print (tmp_rmse)
    
    models.append(model_lgbm)
    rmses.append(tmp_rmse)

(Prediction portion omitted) First submission. The score was 0.909..."Bad!" 
（予測部分は省略）まずは最初のSubmit。スコアの結果は"0.909"・・・「低っ！」

Try to consider other Features. What came up is the idea "Readability = written by easy words", but not sure how to capture 'easy words'... I could say "the easier and easier, the more and more frequently appears". Based on this idea, try to use Countvectorizer. 


他の特徴量を考えてみる。読みやすさ＝「簡単な単語で書かれている」ということかな？とも思うのだが、どうやって「簡単な」を把握すれば良いか分からない・・・「簡単な単語ほど何度も出てくる」と言えなくもないので、単純にCountvectorizerで出現頻度を加えてみる。既存の特徴量と合わせて学習させるため、合算してPandasのデータフレームに追加する。

In [ ]:
countvec = CountVectorizer(stop_words = "english")
cv = countvec.fit_transform(all_df["excerpt"])
dfcv = cv.toarray()
dfcv_sum = pd.DataFrame(np.sum(dfcv,axis=1))
dfcv_sum.columns = ["excerpt_cv"]

all_df = pd.concat([train, test], sort = False).reset_index(drop=True)
all_df['excerpt_len'] = all_df['excerpt'].apply(lambda x : len(x))
all_df['excerpt_wordCnt'] = all_df['excerpt'].apply(lambda x : len(x.split(' ')))
all_df['excerpt_sentCnt'] = all_df['excerpt'].apply(lambda x : len(x.split('.')))
all_df["mean_WordPerSent"] = all_df["excerpt_wordCnt"]/all_df["excerpt_sentCnt"]
all_df["mean_LenPerWord"] = all_df["excerpt_len"]/all_df["excerpt_wordCnt"]

all_df = pd.concat([all_df, dfcv_sum], axis=1)
X_train = all_df[~all_df["target"].isnull()]
X_train = X_train.drop(["url_legal","license","target","excerpt","standard_error"],axis=1)
Y_train = train["target"]
X_train.head()

Try to submit after adding the sum of Countetorizer as a feature. The score becomes 0.99. Slightly improved, but alomost the same...Based on the idea that DIFFICULT sentence includes difficult words which rarely appear, try to add Tfidf.


Countvectorizerの合算値を特徴量に加えて再度Submitしてみる。スコアの結果は「0.900」。微妙に改善したが殆ど変わらない。読みにくい文章にはあまり使われないムズカシイ単語が出てくる、ということも言えるかなという考えで、Tfidfも特徴量に加えてみる。

In [ ]:
tfvec = TfidfVectorizer(stop_words="english")
tfv = tfvec.fit_transform(all_df["excerpt"])
dftfv = tfv.toarray()

dftfv_sum = pd.DataFrame(np.sum(dftfv,axis=1))
dftfv_sum.columns = ["excerpt_tfv"]

all_df = pd.concat([all_df, dfcv_sum, dftfv_sum], axis=1)
X_train = all_df[~all_df["target"].isnull()]
X_train = X_train.drop(["url_legal","license","target","excerpt","standard_error"],axis=1)
Y_train = train["target"]
X_train.head()

Predict again using the newly-added Tfidif sum. The score shows 0.885. Slightly improved again, but still alomost the same...By chance, it may have been bad that I used the easy method to separate using a space or a period in order to count the number of words or sentences. try to separate using the tokenizer.


新たに加えたTfidf合算値も使って再度予測。結果は0.885。ちょっと良くなったかなぁ。でもやはり余り変わらない。単語や分の数を数えるのにスペースで区切ったりピリオドで区切ったりと簡易的な方法を用いたのが良くなかったのかもしれない。tokenizerで区切ってみる。

In [ ]:
all_df['tokenized_excerpt'] = [word_tokenize(p.lower()) for p in all_df["excerpt"]]
all_df['word_count_tk'] = all_df['tokenized_excerpt'].apply(lambda x: len(x))
all_df["avg_LenPerWord"] = all_df["excerpt_len"]/all_df["excerpt_wordCnt"]
all_df['sent_excerpt'] = [nltk.tokenize.sent_tokenize(x) for x in all_df["excerpt"]]
all_df['excerpt_sentCnt'] = all_df['sent_excerpt'].apply(lambda x: len(x))
all_df["avg_WordPerSent"] = all_df["excerpt_wordCnt"]/all_df["excerpt_sentCnt"]

Try to predict again and submit after re-counting words or sentences using Tokenizer. The score shows 0.860. Again, a little bit improved. I'd like to expect more improvement. Reconsider whether my first approach to count words or sentences.. Was it really correct? As for the countvectorizer or tfidfvectorizer as well, would it be better to use them as an array, rather than sum up? 


Tokenizerを使って単語数・文数・平均値等を数えなおし、再度予測をやり直しSubmitしてみる。結果は0.860。また微妙に少し改善。徐々には改善しているものの、もう少し大幅に改善したい。最初に思いついた単語数とか文章数とかの特徴量って、本当に有効だったのだろうか？CountvectorizerとかTfidfvectorizerとかも配列を合算して値として用いているがそのまま配列として使ったほうが良かったりしないか？

In [ ]:
import scipy

countvec = CountVectorizer()
cv = countvec.fit_transform(all_df["excerpt"])

tfvec = TfidfVectorizer(stop_words="english")
tfv = tfvec.fit_transform(all_df["excerpt"])

X = scipy.sparse.hstack((cv, tfv)).tocsr()

X_train = X[:len(train)]
X_test = X[len(train):]
Y_train = train["target"]

X_train = X_train.toarray()
X_test = X_test.toarray()

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

Try to predict and submit again, using Countectorizer and Tfidfvectorizer as an array and not using number of words / sentences. The score shows 0.798. Improvement is the biggest. As expected, it'd be better not to stick to the number of words / sentences. Aiming a bit more improvement, try to optimize the parameters using Optuna.


単語数や文章数は使わずに、CountvectorizerとTfidfvectorizerをそのまま配列として用い、再度予測・submitしてみる。結果は0.798。これまでより改善幅が大きい。やっぱり変に単語数や文章数を用いないほうが良さそう。もう少し改善しないかなぁということでOptunaでパラメーターを最適化してみる。

In [ ]:
models = []
rmses =[]
lgbm_params ={
    "objective":"regression",
    "random_seed":1234,
    "learning_rate":0.05,
    "n_estimators":1000,
    'num_leaves': 64,
    'max_bin': 82,
    'bagging_fraction': 0.7315391015500504,
    'bagging_freq': 3,
    'feature_fraction': 0.41032549973286436,
    'min_data_in_leaf': 13,
    'min_sum_hessian_in_leaf': 4
}

for train_index, val_index in kf.split(X_train):
    XX_train = X_train.iloc[train_index]
    XX_valid = X_train.iloc[val_index]
    YY_train = Y_train.iloc[train_index]
    YY_valid = Y_train.iloc[val_index]
    
    lgbm_train = lgbm.Dataset(XX_train, YY_train)
    lgbm_eval = lgbm.Dataset(XX_valid, YY_valid)
    
    model_lgbm = lgbm.train(lgbm_params,
                           lgbm_train,
                           valid_sets = lgbm_eval,
                           num_boost_round = 100,
                           early_stopping_rounds = 20,
                           verbose_eval = 10,
                           )
    y_pred = model_lgbm.predict(XX_valid, num_iteration = model_lgbm.best_iteration)
    tmp_rmse = np.sqrt(mean_squared_error(YY_valid, y_pred))
    print (tmp_rmse)
    
    models.append(model_lgbm)
    rmses.append(tmp_rmse)

Execute LightGBM again after setting the best parameters found by Optuna.


Optunaで得られたベストなパラメーターをセットし、LightGBMを再度実行。

In [ ]:
preds = []

for model in models:
    pred = model.predict(X_test)
    preds.append(pred)
    
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis =0)

submission = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")
submission["target"] = preds_mean
submission.to_csv("submission.csv", index=False)

The result is 0.779. This is what a begginer like me can do after trial and error.

結果は0.779。初心者の試行錯誤としてはこんなものでしょうか。